<a href="https://colab.research.google.com/github/envomp/2020-Text-Mining/blob/master/week3/word_association_mining_syntacmatic_relations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install conllu estnltk==1.6.0b0

Loeb kõik paarid mällu

In [6]:
import pandas as pd
from estnltk import Text
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET
import ast
from collections import Counter
import time
import json

sample_data = [
    ("Eesti ilukirjandus", "estonian_literature_1990"),
    ("Eesti seadused", "estonian_laws")
]

def read_files_from_dir_with_cache(dir):
    answer = Counter()
    for f in listdir(dir):
        if f.endswith(".json"):
            continue
        cache_location = join(dir, f + ".json")
        if isfile(join(dir, f)):
            if isfile(cache_location): 
                cache = open(cache_location, "r")
                answer += Counter(json.load(cache))
                cache.close()
            else:
                raw_data = Text(formatted_data(join(dir, f))).tag_layer(["words"])
                running_counter = Counter()
                last_word = None
                for word in raw_data.words:
                    if last_word != None:
                        combo = last_word.text + " " + word.text
                        running_counter[combo] += 1
                    last_word = word
                cache = open(cache_location, "w")
                json.dump(dict(running_counter), cache)
                answer += running_counter
                cache.close()
    return answer

def get_data(root):
    if len(root.getchildren()) == 0:
        if root.text:
            return root.text
        else:
            return ""
    data = ""
    for child in list(root.getchildren()):
        data += f"\n{get_data(child)}"
    return data

def formatted_data(filename):
    data = get_data(ET.parse(filename).getroot())
    return data

texts = [
    (
      name, 
      read_files_from_dir_with_cache(dir)
    ) for name, dir in sample_data
]

Uurib süntagmaatilisi seoseid sageduste ja osakaalude alusel

In [29]:
part_two = ['takka', 'tagant', 'kokku', 'ära', 'läbi', 'järel', 'alt', 'valmis', 'valla',
            'eemale', 'maha', 'kõrvale', 'järele', 'kallale', 'lahti', 'alla', 'minema',
            'edasi', 'koos', 'kätte', 'pärale', 'üles', 'esile', 'sekka', 'vahelt', 'külge',
            'tasa', 'tulema', 'pealt', 'vastu', 'kaasas', 'kaasa', 'ilma', 'taha', 'peale',
            'kinni', 'peal', 'ringi', 'täis', 'välja', 'püsti', 'laiali', 'ette', 'ühte',
            'lahku', 'tagasi', 'juurde', 'ligi', 'vahele', 'ülal', 'kõrvalt', 'üle', 'otsa',
            'sisse', 'üleval', 'mööda', 'ühes', 'pihta', 'taga', 'ümber']

for name, counter in texts:
    print(f"{name}: enim kasutatavad  ühendverbid:")
    container = []
    total = 0
    for item, reps in counter.most_common(100000):
        text = Text(item).tag_layer(["morph_analysis"])
        if len(text.words) != 2:
            continue
        A = text.words[0]
        B = text.words[1]
        if len(A.partofspeech) == 1 and len(A.form) == 1 and len(B.partofspeech) == 1 and len(B.form) == 1:
            AP = A.partofspeech[0]
            BP = B.partofspeech[0]
            AF = A.form[0]
            BF = B.form[0]
            if     AP == "D" and BP == "D" \
                or AP == "D" and BP == "S" \
                or AP == "D" and BP == "A" \
                or AP == "D" and BP == "V" \
                or AP == "A" and BP == "S" \
                or AP == "A" and BP == "A" \
                or AP == "A" and BP == "V" and AF.endswith('tr') \
                or AP == "A" and BP == "V" and AF.endswith('n') \
                or AP == "S" and BP == "S" and AF.endswith('g') \
                or AP == "S" and BP == "V" and AF.endswith('n') \
                or AP == "N" and BP == "S" and BF.endswith('p'):
                ending = ""
                if A.text in part_two:
                    ending = " <----------------------------------------------------------"
                total += reps
                container.append((item, reps, ending))
    
    for i in range(min(200, len(container))):
        item, reps, ending = container[i]
        print(f"{item} - {reps} - {round(100 * reps / total, 2)}%{ending}")
    print("\n\n")

Eesti ilukirjandus: enim kasutatavad  ühendverbid:
kogu aeg - 1163 - 0.55%
ikka veel - 1144 - 0.54%
enam ei - 1007 - 0.48%
nii palju - 961 - 0.45%
siis ei - 805 - 0.38%
ka ei - 652 - 0.31%
siis oli - 610 - 0.29%
küll ei - 501 - 0.24%
siis veel - 462 - 0.22%
mitte ainult - 436 - 0.21%
juba ammu - 424 - 0.2%
liiga palju - 414 - 0.2%
mina ei - 403 - 0.19%
ju ka - 377 - 0.18%
lihtsalt ei - 371 - 0.18%
veel kord - 362 - 0.17%
nii väga - 358 - 0.17%
alles siis - 340 - 0.16%
ka mitte - 337 - 0.16%
Nüüd oli - 326 - 0.15%
välja ei - 322 - 0.15% <----------------------------------------------------------
veel ei - 307 - 0.15%
hea meel - 296 - 0.14%
ju ei - 295 - 0.14%
kõige rohkem - 295 - 0.14%
ära ei - 291 - 0.14% <----------------------------------------------------------
mitte kunagi - 290 - 0.14%
viimasel ajal - 287 - 0.14%
väga hästi - 283 - 0.13%
siis ka - 273 - 0.13%
ka siis - 272 - 0.13%
nii edasi - 271 - 0.13%
kaks korda - 261 - 0.12%
mees oli - 256 - 0.12%
isegi ei - 256 - 0.12%
tõesti

Ühendverbid, mis algasid õige esimese sõnaga aga polnud EKSS loendis olid: kaasa võtma ja üle teostama?